# Electrochemical Impedance Spectroscopy - Interactive Simulations

This notebook/webpage contains an set of interactive visualizations of the frequency dependent impedance (we'll use the symbol Z for impedance) of a few equivalent circuit models commonly encountered in electrochemical impedance spectroscopy.

The impedance data is generally visualized using different 2D representations such as:
* **Complex plane plot** (or a Nyquist plot) where the real ($Z_{re}$) and imaginary ($Z_{im}$) part of the impedance is displayed. Most often the negative of the imaginary that is plotted on the y-axis (*i.e.* - $Z_{im}$) in such a plot.

* a **Bode plot**, which is a plot with the logarithm of the frequency ($log\,f$) on the x-axis in one of the following combinations:
    * the modulus of the logarithm ($log_{10}$) of the impedance ($Z_{mod}$) and the phase angle ($\phi$) *vs* $log\ f$
    * $Z_{re}$ and $-Z_{im}$ *vs* $log\,f$
    * $log\,Z_{re}$ and $log\,|Z_{im}|$ *vs* $log\,f$

The circuits are described using a text string that can (unambiguously) describe the way the circuit is connected. It is closely following that suggested by B. Boukamp in his [Circuit Description Code (CDC)](https://www.utwente.nl/en/tnw/ims/publications/downloads/CDC_Explained.pdf).

The image below shows a circuit diagram illustrating the well-known Randles circuit, where $R_{\Omega}$ is the ohmic resistance (e.g. the uncompensated electrolyte resistance in a three-electrode cell), $R_{ct}$ is the charge transfer resistance, C is the double-layer capacitance, and W is a semi-infinite Warburg diffusion element.

![Randles Circuit](../figures/EquivalentCircuitRandles.png)

Using the CDC we can describe the Randles circuit as "R(C\[RW\])". This type of string representation is helpful when one wants to instruct a computer program what circuit to build or select for fitting of a model to experimental impedance data, and also useful to discuss different circuits in text.


** Things to implement **

* move all impedance functions and interact wrappers to a script that is imported into notebook to unclutter, but add MathJax equations for all cases
* hmm, now I am thinking that this should be split over several notebooks/html docs...
* a R(RC) circuit
* a R(C[RW]) circuit where W is a semi-infite Warburg diffusion element
* a R(C[RW]) circuit where W is a 1D finite-length diffusion element with a blocking boundary (to illustrate response in thin-layer cell)
* a R(C[RW]) circuit in the context of an RDE experiment, in this case it is a finite-length diffusion element with a transmissive boundary
* a Transmission Line Circuit Model / Porous Electrode example (maybe this should be a separate notebook/html)
* a concentration profile in EIS experiment where frequency can be varied to illustrate "probing depth" / penetration length in EIS

In [3]:
import nbinteract as nbi# importing nbinteract package
#            Binder spec       Name of the notebook
#nbi.publish('my_binder_spec', 'my_notebook.ipynb') # for generating the html of this notebook
# binder spec = {username}/{repo}/{branch}
# so here it would be 'hjelmj/OpenEIS/master'...

In [2]:
# import functions for interactive visualizations
from EIS_interactive import RC_interactive, R_RC_interactive
# set up notebook for inline plots
%matplotlib inline
# switch to svg output
%config InlineBackend.figure_format = 'svg'
# select matplotlib stylesheet
plt.style.use('ggplot')

## Adding up Impedances

The main three rules to remember are the following:

* Series connected impedances: $Z = Z_1 + Z_2 + Z_3 + ... + Z_i$  
  \<place image here>
* Parallel connected impedances: $\frac{1}{Z} = \frac{1}{Z_1} + \frac{1}{Z_2} + ... + \frac{1}{Z_i}$  
  \<place image here>
* When the circuit contains both parallel and series connected elements:  
  \<place image here>
  
*need to make illustration using draw.io similar to that in EIS lecture...*

## A series connected R and C element, an RC circuit
The impedance of a resistor, $Z_R = R$, and the impedance of a capacitor is $Z_C =  


## A simple R(RC) circuit
The following interactive simulation is for a simple R(RC) circuit as illustrated below:

The impedance of a resistor, $Z_R$ = R and the impedance of a capacitor $Z_C = \dfrac{1}{j \omega C}$ and the impedance of the R(RC) circuit becomes:

$$Z_{R(RC)} = R_0 +   \dfrac{1}{ \left ( \dfrac{1}{R_{1}} + \dfrac{1}{(j\omega)C_1} \right )}  $$

An example of a Python function that returns the impedance of a R(RC) circuit is given below.

In [3]:
# define function that returns impedance of -R-(RC)- circuit

def Z_R_RC(R0, R1, C1, w):
    """
    Returns the impedance of a -R-(RC)- circuit.
    
    Parameters:
    -----------
    R0 : series resistance (Ohmic resistance) of circuit [Ohm]
    R1 : resistance of parallel connected circuit element [Ohm]
    C1 : capacitance of parallel connected circuit element [Farad]
    w : angular frequency, accepts an array as well as a single number [rad s^-1]
    
    Returns:
    --------
    The frequency dependent impedance as a complex number. [Ohm]
    """
    Z_R0 = R0
    Z_R1 = R1
    Z_C1 = -1j/(w*C1) # capacitive reactance
    Z_RC = 1/(1/Z_R1 + 1/Z_C1) # parallel connection
    return Z_R0 + Z_RC # Z_R0 and Z_RC connected in series

In [4]:
# generate an angular frequency range
w = np.logspace(7,0,7*10)

### Interactive Plot - R(RC) Circuit

1. Vary the Ohmic resistance (R_0 here) - foes it influence the shape or summit frequency of the semi-circle?
2. 
* add some questions / instructions for students / get feedback from dorhoff and lncl...

In [6]:
R_RC_interactive()

interactive(children=(FloatSlider(value=10.0, description='$R_0$', max=30.0, readout_format='.1f', step=1.0), FloatSlider(value=20.0, description='$R_1$', max=30.0, min=5.0, readout_format='.1f', step=1.0), FloatSlider(value=1e-05, continuous_update=False, description='$C_1$', max=0.001, min=1e-06, readout_format='.2e', step=1e-06), Output()), _dom_classes=('widget-interact',))

## Electrochemical Impedance of Randles Circuit with Semi-infinite diffusion element
This case corresponds to the situation in a electrochemical three electrode set up, where a dissolved redox active molecule or ion is present in an solution with supporting electrolyte. The concentration (or ionic strength) of the supporting electrolyte is typically 50-100 times larger than the total concentration of the redox couple.

\<put illustration of electrode surface + processes here...>

In [7]:
# define function that returns the impedance of a Randles circuit with a semi-infinite diffusion element

def Z_Randles_si(w,R0=5, Rct=50, C=1e-5, n=1, Dox=1e-5, Dred=1e-5, Cox=1e-6, Cred=1e-6, A=1, T=298):
    """
    docstring
    """
    Z_R0 = R0 # series resistance (uncompensated electrolyte resistance)
    sigma = ((R*T)/(n**2 * F**2 * A * np.sqrt(2)))*(1/(np.sqrt(Dox)*Cox) + 1/(np.sqrt(Dred)*Cred)) # Warburg coefficient
    Z_W = sigma/w**0.5 - 1j*sigma/w**0.5 # Warburg impedance, semi-infinite planar diffusion
    Z_f = Rct + Z_W # impedance of faradaic branch
    Z_C = -1j/(w*C1) # capacitive reactance - impedance of capacitor (models double-layer capacitance)            
    Z_parallel = 1/(1/Z_C + 1/Z_f)
    Z_total = Z_R0 + Z_parallel
    return Z_total

def Randles_si_plot(np.logspace(7,0,7*10),R0=5, Rct=50, C=1e-5, n=1, Dox=1e-5, Dred=1e-5, Cox=1e-6, Cred=1e-6, A=1, T=298):
    Z = Z_Randles_si(w, R0=R0, Rct=Rct, C=C, n=n, Dox=Dox, Dred=Dred, Cox=Cox, Cred=Cred, A=A, T=T)
    #set up a figure canvas with two plot areas (sets of axes)
    fig,ax = plt.subplots(nrows=1, ncols=1, figsize=(8,4))
    #add a Nyquist plot (first plot)
    ax.plot(Z.real, -1*Z.imag, marker='o',ms=5, mec='b', mew=0.7, mfc='none')
    ax.set_xlim(0,60)
    ax.set_ylim(0,60)
    ax.set_aspect('equal')
    ax.set_xlabel('Z$_{re}$ [$\Omega$]')
    ax.set_ylabel('-Z$_{im}$ [$\Omega$]')
    plt.show()
    
def Randles_si_interactive():
    interact(Randles_si_plot, 
             R0=FloatSlider(description='$R_0$', min=0, max=30, step=1, value=10, orientation='horizontal', 
                            readout=True, readout_format='.1f', continuous_update=True),
             Rct=FloatSlider(description='$R_1$', min=5, max=30, step=1, value=20, orientation='horizontal', 
                            readout=True, readout_format='.1f', continuous_update=True),
             C=FloatSlider(description='$C_1$', min=1e-6, max=1e-3, step=1e-6, value=1e-5,orientation='horizontal', 
                            readout=True, readout_format='.2e', continuous_update=False))

SyntaxError: invalid syntax (<ipython-input-7-50421c6cd329>, line 16)

## Finite-length diffusion with blocking boundary - The impedance of a thin layer cell

Try to add both impedance plot and a second plot illstrating wall<->electrode gap

## Finite-length diffusion with transmissive boundary - The impedance of a rotating electrode

Try to add panel with four plots:

 1. voltammetry / i-V curve from RDE
 2. impedance plot normalized impedance
 3. impedance plot absolute impedance
 4. Bode plot

Using the `version information` extension by J.R. Johansson to display software version used to create this notebook (see [here](https://nbviewer.jupyter.org/github/jrjohansson/version_information/blob/master/example.ipynb) for install and usage instructions):

In [ ]:
%load_ext version_information

In [ ]:
%version_information numpy, matplotlib, ipywidgets